<a href="https://colab.research.google.com/github/Wavelydavely/World_Happiness_Report/blob/test_lili/backward_elimination.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [140]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder, LabelEncoder

from sklearn.linear_model import LinearRegression

# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt

#  Import and read the csv.
import pandas as pd 
happiness_df = pd.read_csv("World_Happiness_Report_2021.csv")
happiness_df.head(10)

,Country,Year,World_Region,Happiness_Score,Economic_Production,Social_Support,Life_Expectancy,Freedom,Generosity,Perceptions_of_Corruption
0,Finland,2021,Western Europe,7.842,10.775,0.954,72.0,0.949,-0.098,0.186
1,Denmark,2021,Western Europe,7.620,10.933,0.954,72.7,0.946,0.030,0.179
2,Switzerland,2021,Western Europe,7.571,11.117,0.942,74.4,0.919,0.025,0.292
3,Iceland,2021,Western Europe,7.554,10.878,0.983,73.0,0.955,0.160,0.673
4,Netherlands,2021,Western Europe,7.464,10.932,0.942,72.4,0.913,0.175,0.338
5,Norway,2021,Western Europe,7.392,11.053,0.954,73.3,0.960,0.093,0.270
6,Sweden,2021,Western Europe,7.363,10.867,0.934,72.7,0.945,0.086,0.237
7,Luxembourg,2021,Western Europe,7.324,11.647,0.908,72.6,0.907,-0.034,0.386
8,New Zealand,2021,North America and ANZ,7.277,10.643,0.948,73.4,0.929,0.134,0.242
9,Austria,2021,Western Europe,7.268,10.906,0.934,73.3,0.908,0.042,0.481


In [141]:
# Generate our categorical variable lists
happiness_cat = happiness_df.dtypes[happiness_df.dtypes == "object"].index.tolist()

In [143]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_happiness_df = pd.DataFrame(enc.fit_transform(happiness_df[happiness_cat]))

# Add the encoded variable names to the dataframe
encode_happiness_df.columns = enc.get_feature_names(happiness_cat)
encode_happiness_df.head()

,Country_Afghanistan,Country_Albania,Country_Algeria,Country_Argentina,Country_Armenia,Country_Australia,Country_Austria,Country_Azerbaijan,Country_Bahrain,Country_Bangladesh,Country_Belarus,Country_Belgium,Country_Benin,Country_Bolivia,Country_Bosnia and Herzegovina,Country_Botswana,Country_Brazil,Country_Bulgaria,Country_Burkina Faso,Country_Burundi,Country_Cambodia,Country_Cameroon,Country_Canada,Country_Chad,Country_Chile,Country_China,Country_Colombia,Country_Comoros,Country_Congo (Brazzaville),Country_Costa Rica,Country_Croatia,Country_Cyprus,Country_Czech Republic,Country_Denmark,Country_Dominican Republic,Country_Ecuador,Country_Egypt,Country_El Salvador,Country_Estonia,Country_Ethiopia,...,Country_Singapore,Country_Slovakia,Country_Slovenia,Country_South Africa,Country_South Korea,Country_Spain,Country_Sri Lanka,Country_Swaziland,Country_Sweden,Country_Switzerland,Country_Taiwan Province of China,Country_Tajikistan,Country_Tanzania,Country_Thailand,Country_Togo,Country_Tunisia,Country_Turkey,Country_Turkmenistan,Country_Uganda,Country_Ukraine,Country_United Arab Emirates,Country_United Kingdom,Country_United States,Country_Uruguay,Country_Uzbekistan,Country_Venezuela,Country_Vietnam,Country_Yemen,Country_Zambia,Country_Zimbabwe,World_Region_Central and Eastern Europe,World_Region_Commonwealth of Independent States,World_Region_East Asia,World_Region_Latin America and Caribbean,World_Region_Middle East and North Africa,World_Region_North America and ANZ,World_Region_South Asia,World_Region_Southeast Asia,World_Region_Sub-Saharan Africa,World_Region_Western Europe
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [144]:
# Merge one-hot encoded features and drop the originals
happiness_df = happiness_df.merge(encode_happiness_df,left_index=True, right_index=True)
happiness_df = happiness_df.drop(happiness_cat,1)
happiness_df.head()

,Year,Happiness_Score,Economic_Production,Social_Support,Life_Expectancy,Freedom,Generosity,Perceptions_of_Corruption,Country_Afghanistan,Country_Albania,Country_Algeria,Country_Argentina,Country_Armenia,Country_Australia,Country_Austria,Country_Azerbaijan,Country_Bahrain,Country_Bangladesh,Country_Belarus,Country_Belgium,Country_Benin,Country_Bolivia,Country_Bosnia and Herzegovina,Country_Botswana,Country_Brazil,Country_Bulgaria,Country_Burkina Faso,Country_Burundi,Country_Cambodia,Country_Cameroon,Country_Canada,Country_Chad,Country_Chile,Country_China,Country_Colombia,Country_Comoros,Country_Congo (Brazzaville),Country_Costa Rica,Country_Croatia,Country_Cyprus,...,Country_Singapore,Country_Slovakia,Country_Slovenia,Country_South Africa,Country_South Korea,Country_Spain,Country_Sri Lanka,Country_Swaziland,Country_Sweden,Country_Switzerland,Country_Taiwan Province of China,Country_Tajikistan,Country_Tanzania,Country_Thailand,Country_Togo,Country_Tunisia,Country_Turkey,Country_Turkmenistan,Country_Uganda,Country_Ukraine,Country_United Arab Emirates,Country_United Kingdom,Country_United States,Country_Uruguay,Country_Uzbekistan,Country_Venezuela,Country_Vietnam,Country_Yemen,Country_Zambia,Country_Zimbabwe,World_Region_Central and Eastern Europe,World_Region_Commonwealth of Independent States,World_Region_East Asia,World_Region_Latin America and Caribbean,World_Region_Middle East and North Africa,World_Region_North America and ANZ,World_Region_South Asia,World_Region_Southeast Asia,World_Region_Sub-Saharan Africa,World_Region_Western Europe
0,2021,7.842,10.775,0.954,72.0,0.949,-0.098,0.186,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,2021,7.620,10.933,0.954,72.7,0.946,0.030,0.179,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,2021,7.571,11.117,0.942,74.4,0.919,0.025,0.292,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,2021,7.554,10.878,0.983,73.0,0.955,0.160,0.673,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,2021,7.464,10.932,0.942,72.4,0.913,0.175,0.338,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [145]:
# Split our preprocessed data into our features and target arrays
y = happiness_df["Happiness_Score"] # y = dependent = ladder score
X = happiness_df.drop(["Happiness_Score"],axis=1).values # X = independent = any other variable

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.2)

In [146]:
# preprocess strings. one hot in coder
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [147]:
# Introduce regressor
regressor = LinearRegression()
regressor.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [148]:
# Create vector of predictions
y_pred = regressor.predict(X_test)

In [149]:
print(f"Prediction: {y_pred}")

Prediction: [4.483908   5.38895169 6.02045702 5.7604076  7.28943966 4.10887097
 4.07366768 4.08447135 4.40038078 4.27549496 6.02819397 5.84738923
 5.78307078 6.15257334 6.00190076 4.56432964 5.03626603 6.76726017
 7.07444662 5.86066922 6.34482973 7.07670988 5.51989164 7.02774082
 5.88686776 4.80953779 4.36431982 6.79084896 6.12370329 4.13661803]


In [150]:
# Show actual value with predicted value
regressor_diff = pd.DataFrame({'Actual value': y_test, 'Predicted value': y_pred})
real_values = regressor_diff.sort_values('Actual value', ascending=False)
real_values

,Actual value,Predicted value
7,7.324,7.289440
8,7.277,7.076710
16,7.064,7.074447
22,6.602,7.027741
24,6.561,6.152573
26,6.491,6.790849
33,6.331,6.028194
37,6.255,6.001901
40,6.180,6.344830
43,6.166,6.123703


In [151]:
#Model Evaluation
from sklearn import metrics
MAE = metrics.mean_absolute_error(y_test, y_pred)
MSE = metrics.mean_squared_error(y_test, y_pred)
RMSE = np.sqrt(metrics.mean_squared_error(y_test, y_pred))
print('R squared: {:.2f}'.format(regressor.score(X, y)))
print('Mean Absolute Error:', MAE)
print('Mean Square Error:', MSE)
print('Root Mean Square Error:', RMSE)

R squared: 0.96
Mean Absolute Error: 0.3604509841727827
Mean Square Error: 0.22304200018817413
Root Mean Square Error: 0.4722732262029832


Backward Elimination 

In [162]:
import statsmodels.api as smf  

In [163]:
#Avoiding the dummy variable trap:  
X = X[:, 1:]  

In [164]:
 # Add a column of ones in our matrix of features
X = np.append(arr = np.ones((149,1)).astype(int), axis=1, values=X)

In [170]:
# Create matrix of optimal features (X_optimal) these features are statistically significant(have high impact on ladder score)
X_optimal = X[:, [0, 1, 2, 3, 4, 5]]
regressor_opt = smf.OLS(endog = y, exog = X_optimal).fit()


In [171]:
# Summarize 
regressor_opt.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        Happiness_Score   R-squared:                       0.748
Model:                            OLS   Adj. R-squared:                  0.740
Method:                 Least Squares   F-statistic:                     85.07
Date:                Tue, 10 Aug 2021   Prob (F-statistic):           4.32e-41
Time:                        16:38:26   Log-Likelihood:                -118.75
No. Observations:                 149   AIC:                             249.5
Df Residuals:                     143   BIC:                             267.5
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.1559      0.456     -6.924      0.000      -4.057      -2.255
x1             0.3093      0.087      3.569      0.000       0.138       0.481
x2             2.1679      0.659      3.289      0.001       0.865       3.471
x3             0.0339      0.013      2.534      0.012       0.007       0.060
x4             2.2870      0.482      4.743      0.000       1.334       3.240
x5             0.4916      0.319      1.541      0.125      -0.139       1.122
==============================================================================
Omnibus:                        9.397   Durbin-Watson:                   1.676
Prob(Omnibus):                  0.009   Jarque-Bera (JB):                9.328
Skew:                          -0.561   Prob(JB):                      0.00943
Kurtosis:                       3.495   Cond. No.                     1.00e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [ ]:
# if p value > 0.05 then remove said index by removing index from X_optimal 
